In [1]:
import pandas as pd


from AutoMLWrapper import AutoMLWrapper


/home/max/miniconda3/envs/AutoML/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-11-21 13:28:48.625562: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-21 13:28:49.191301: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-11-21 13:28:49.191380: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugi

Using TensorFlow backend


In [2]:
df = pd.read_csv('data/glass.csv')

In [3]:
wrapper = AutoMLWrapper('autogluon')

#wrapper.initialize(df, 'Type', task_type='classification', data_type='tabular', problem_type='multiclass')

wrapper.train(data=df, target_column='Type', task_type='classification', data_type='tabular', problem_type='multiclass')

No path specified. Models will be saved in: "AutogluonModels/ag-20231121_122851/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20231121_122851/"
AutoGluon Version:  0.8.2
Python Version:     3.8.18
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #37~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Mon Oct  9 15:34:04 UTC 2
Disk Space Avail:   108.25 GB / 483.95 GB (22.4%)
Train Data Rows:    214
Train Data Columns: 9
Label Column: Type
Preprocessing data ...
Train Data Class Count: 6
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    26918.98 MB
	Train Data (Original)  Memory Usage: 0.02 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerat

	0.6744	 = Validation score   (accuracy)
	0.01s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
No improvement since epoch 6: early stopping
	0.7674	 = Validation score   (accuracy)
	0.78s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: LightGBMXT ...
	0.8605	 = Validation score   (accuracy)
	0.51s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: LightGBM ...
	0.8372	 = Validation score   (accuracy)
	0.38s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: RandomForestGini ...


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:5                                                                                    │
│                                                                                                  │
│   2                                                                                              │
│   3 #wrapper.initialize(df, 'Type', task_type='classification', data_type='tabular', problem     │
│   4                                                                                              │
│ ❱ 5 wrapper.train(data=df, target_column='Type', task_type='classification', data_type='tabu     │
│   6                                                                                              │
│                                                                                                  │
│ /home/max/Documents/Masterarbeit/AutoML/AutoMLWrapper/AutoMLWrapper.py:60 in train               │
│                                                                                                  │
│   57 │   │   if not self.is_initialized:                                                         │
│   58 │   │   │   self.initialize(data, target_column, task_type, data_type, problem_type)        │
│   59 │   │                                                                                       │
│ ❱ 60 │   │   self.library._train_model(data, target_column, hyperparameters)                     │
│   61 │                                                                                           │
│   62 │   #-----------------------------------------------------------------------------------    │
│   63 │   def evaluate(self, test_data: Any) -> float:                                            │
│                                                                                                  │
│ /home/max/Documents/Masterarbeit/AutoML/AutoMLWrapper/AutoGluon/AutoGluonWrapper.py:118 in       │
│ _train_model                                                                                     │
│                                                                                                  │
│   115 │   │   │   │   **(self.config.get_params_constructor_by_key('TimeSeriesPredictor') or {   │
│   116 │   │   │   )                                                                              │
│   117 │   │                                                                                      │
│ ❱ 118 │   │   self.model.fit(                                                                    │
│   119 │   │   │   data,                                                                          │
│   120 │   │   │   **(self.config.get_params_fit_by_key('TimeSeriesPredictor' if self.problem_t   │
│   121 │   │   │   │   │   │   │   │   │   │     else 'TabularPredictor' if self.problem_type =   │
│                                                                                                  │
│ /home/max/miniconda3/envs/AutoML/lib/python3.8/site-packages/autogluon/core/utils/decorators.py: │
│ 31 in _call                                                                                      │
│                                                                                                  │
│   28 │   │   @functools.wraps(f)                                                                 │
│   29 │   │   def _call(*args, **kwargs):                                                         │
│   30 │   │   │   gargs, gkwargs = g(*other_args, *args, **kwargs)                                │
│ ❱ 31 │   │   │   return f(*gargs, **gkwargs)                                                     │
│   32 │   │                                                                                       │
│   33 │   │   return _call                                                                        │
│   34                                                       